In [12]:
# gradient boosting
import os
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor
import networkx as nx

# https://catboost.ai/en/docs/concepts/tutorials
# https://youtu.be/usdEWSDisS0

In [13]:
NUMBER_NODES = 7

def load_data():
    train_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_train.csv'))
    val_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_val.csv'))
    test_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_test.csv'))

    featuresNumber = (NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 
    def get_tuple_tensor_dataset(row):
        X = row[0 : featuresNumber].astype('int32')
        Y = row[featuresNumber + 1: ].astype('int32') # Inclui a banda otima na posicao 0
        return X, Y

    train_dataset = list(map(get_tuple_tensor_dataset, train_df.to_numpy()))
    val_dataset = list(map(get_tuple_tensor_dataset, val_df.to_numpy()))
    test_dataset = list(map(get_tuple_tensor_dataset, test_df.to_numpy()))

    X = []
    Y = []
    for x, y in train_dataset:
        X.append(x)
        Y.append(y)
    x_train = np.array(X)
    y_train = np.array(Y)

    X = []
    Y = []
    for x, y in test_dataset:
        X.append(x)
        Y.append(y)
    x_test = np.array(X)
    y_test = np.array(Y)

    X = []
    Y = []
    for x, y in val_dataset:
        X.append(x)
        Y.append(y)
    x_val = np.array(X)
    y_val = np.array(Y)

    x_train = np.concatenate((x_train, x_val))
    y_train = np.concatenate((y_train, y_val))


    return x_train, y_train, x_test, y_test

In [14]:
x_train, y_train, x_test, y_test = load_data()

In [15]:
cat_features = list(range(0, x_test.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [16]:
model = CatBoostRegressor(objective='MultiRMSE', verbose=100)
# model = CatBoostRegressor(objective='MultiRMSEWithMissingValues', verbose=100)
model.fit(x_train, y_train, eval_set=(x_test, y_test), cat_features=cat_features, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4.9685372	test: 5.0103356	best: 5.0103356 (0)	total: 17ms	remaining: 17s
100:	learn: 4.2595071	test: 4.6323238	best: 4.6323238 (100)	total: 1.13s	remaining: 10.1s
200:	learn: 3.9160863	test: 4.5308368	best: 4.5305104 (199)	total: 2.15s	remaining: 8.54s
300:	learn: 3.6589244	test: 4.4920674	best: 4.4920674 (300)	total: 3.23s	remaining: 7.49s
400:	learn: 3.4594657	test: 4.4632432	best: 4.4584854 (380)	total: 4.38s	remaining: 6.54s
500:	learn: 3.2926053	test: 4.4579057	best: 4.4513965 (466)	total: 5.46s	remaining: 5.43s
600:	learn: 3.1553447	test: 4.4583899	best: 4.4513965 (466)	total: 6.65s	remaining: 4.42s
700:	learn: 3.0330676	test: 4.4716848	best: 4.4513965 (466)	total: 7.88s	remaining: 3.36s
800:	learn: 2.9279096	test: 4.4923077	best: 4.4513965 (466)	total: 9.09s	remaining: 2.26s
900:	learn: 2.8332496	test: 4.5102409	best: 4.4513965 (466)	total: 10.3s	remaining: 1.13s
999:	learn: 2.7492686	test: 4.5257821	best: 4.4513965 (466)	total: 11.5s	remaining: 0us

bestTest = 4.45139

In [17]:
model.tree_count_

467

In [18]:
cat_features

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [19]:
# model.calc_feature_statistics(x_train, y_train, plot=True)

In [20]:
def count_repeats(output):
    counts = np.unique(np.round(output))
    repeated = NUMBER_NODES - counts.shape[0]
    return repeated

def get_valid_pred(pred):
    valid = np.ones(NUMBER_NODES)
    labels = np.arange(0, NUMBER_NODES)
    for i in labels:
        min_value = np.amin(pred)
        min_idx, = np.where(pred == min_value)
        min_idx = min_idx[0]
        pred[min_idx] = 100
        valid[min_idx] = i
    return valid
    
def get_bandwidth(Graph, nodelist):
    Graph = nx.Graph(Graph)
    if not Graph.edges:
        return 0
    if nodelist.all() != None:
        L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    else:
        L = nx.laplacian_matrix(Graph)
    x, y = np.nonzero(L)
    return (x-y).max()

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj

In [21]:
import time

pred = model.predict(x_test)

sumTest_original = []
sumTest_pred = []
sumTest_true = []

count = 0
cases_with_repetition = 0

start = time.time()
for i in range(len(pred)):

    output = pred[i]

    quantity_repeated = count_repeats(np.round(output))
    # print('Pred: ', output)
    # print('True: ', y_test[i])
    if quantity_repeated != 0:
        cases_with_repetition += 1
    output = get_valid_pred(output)
    # print('Pred valid: ', output)
    count += quantity_repeated

    # print("Bandwidth")
    graph = getGraph(x_test[i])
    original_band = get_bandwidth(graph, np.array(None))
    sumTest_original.append(original_band)

    pred_band = get_bandwidth(graph, output)
    sumTest_pred.append(pred_band)

    true_band = get_bandwidth(graph, y_test[i])
    sumTest_true.append(true_band)
    # print("Bandwidth")
    # print(original_band)
    # print(pred_band)
    # print(true_band)
end = time.time()

In [22]:
print('Quantidade de rótulos repetidos, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 6 - ', count)
print('Quantidade de saídas com repetição, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 1 - ', cases_with_repetition)
test_length = pred.shape[0]
print('Test length - ', test_length)
print('Time - ', (end - start)/test_length)
print("Bandwidth mean±std")
print(f'{np.mean(sumTest_original)}±{np.std(sumTest_original)}')
print("Pred bandwidth mean±std")
print(f'{np.mean(sumTest_pred)}±{np.std(sumTest_pred)}')
print("True bandwidth mean±std")
print(f'{np.mean(sumTest_true)}±{np.std(sumTest_true)}')

Quantidade de rótulos repetidos, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 6 -  210
Quantidade de saídas com repetição, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 1 -  62
Test length -  63
Time -  0.004174592002989754
Bandwidth mean±std
5.904761904761905±0.2935435239509036
Pred bandwidth mean±std
4.73015873015873±0.9953279623232306
True bandwidth mean±std
3.1904761904761907±0.7095078297976829
